In [2]:
import numpy as np
import pandas as pd
import re
# make http requests and webscrap
import requests

# Plotting Libraries
import seaborn as sb
from matplotlib import pyplot as plt

# text processing
import string
import nltk
from nltk.corpus import stopwords

In [2]:
#!pip install arxiv bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=2180956a8099813bfa90c66d492160b8cbb2aa729303a13182d8df1090201f9e
  Stored in directory: /Users/shawnyou/Library/Caches/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=587d0fca9da1d76000a5dc1a564c052dc7c0aaefabcb8012114e4cd6053774b4
  Stored in directory: /Users/shawnyou/Library/Caches/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built bs4 sgmllib3k
You should consider upgrading via the '/Users/shawnyou/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from bs4 import BeautifulSoup
import arxiv

In [3]:
def get_snarXiv_abstracts(n_queries = 1):
    # make a GET request to the snarXiv page
    both = []
    abstracts = []
    titles = []
    for n in range(n_queries):
        with requests.get('http://snarxiv.org/') as response:
            # Let's scrap the html document of the page
            soup = BeautifulSoup(response.content)
            # The abstracts are inside <p> tags which are
            # inside of <div class = "meta"> tags.
            abstracts.extend( map(lambda x:x.text,soup.select("dd > div.meta > p")) )
            titles.extend( map(lambda x:x.text,soup.select("dd > div.meta > div.list-title")) )
    both.append(abstracts)
    both.append(titles)
    return both

# Function to create a dataframe with the snarXiv abstracts
def make_snarXiv_dataframe(n_queries = 1):
    summaries = get_snarXiv_abstracts(n_queries = n_queries)
    abstracts = summaries[0]
    titles = summaries[1]
    summaries_length = len(abstracts)
    labels = ['snarXiv']*summaries_length
    return pd.DataFrame(
        data = {
            'summary':abstracts,
            'title':titles,
            'label':labels
        }
    ).rename_axis('id')  

In [4]:
#make_snarXiv_dataframe(n_queries = 40)

In [5]:
snar_data = make_snarXiv_dataframe(100)

In [6]:
snar_data['title'] = snar_data['title'].str[8:]

In [10]:
#snar_data.to_csv('snarxiv_data.csv', index = False)

In [13]:
snar_data['title'] = snar_data['title'].str.rstrip('\n')

In [14]:
snar_data

,summary,title,label
id,,,
0,We make contact with particle fluctuations in ...,Discrete Perturbation Theory in Analyzing E_7 ...,snarXiv
1,We illustrate that abelian modular forms are r...,Minimal Approaches to the Mu Problem,snarXiv
2,We make contact between Geometric Langlands-du...,Flow Equations in Type IIB Strings Deformed by...,snarXiv
3,"Recently, little work was done on the alternat...",Bounding S-dualities in Chiral Jackiw-Teitelbo...,snarXiv
4,We confirm that gravitational waves in a non-S...,Causality Constraints in Conformal CFTs as To...,snarXiv
...,...,...,...
495,"Recently, interesting progress was made analyz...",Firewalls Using Sleptons,snarXiv
496,"Using considering Strassler's equations, we bo...",Anyons,snarXiv
497,"Using explaining bit threads, we formulate an ...",Line Bundles on Non-compact Superspace Using D...,snarXiv


In [15]:
snar_data.to_csv('snarxiv_data.csv', index = False)

In [21]:
test = pd.read_csv('snarxiv_data.csv')

In [22]:
test

,summary,title,label
0,We make contact with particle fluctuations in ...,Discrete Perturbation Theory in Analyzing E_7 ...,snarXiv
1,We illustrate that abelian modular forms are r...,Minimal Approaches to the Mu Problem,snarXiv
2,We make contact between Geometric Langlands-du...,Flow Equations in Type IIB Strings Deformed by...,snarXiv
3,"Recently, little work was done on the alternat...",Bounding S-dualities in Chiral Jackiw-Teitelbo...,snarXiv
4,We confirm that gravitational waves in a non-S...,Causality Constraints in Conformal CFTs as To...,snarXiv
...,...,...,...
495,"Recently, interesting progress was made analyz...",Firewalls Using Sleptons,snarXiv
496,"Using considering Strassler's equations, we bo...",Anyons,snarXiv
497,"Using explaining bit threads, we formulate an ...",Line Bundles on Non-compact Superspace Using D...,snarXiv
498,"Recently, Strassler deduced that a B-type bran...","B-mesons, the Extension of WZW NCFTs Living on...",snarXiv


In [21]:
test.head(20)

,summary,title,label
0,We make contact with particle fluctuations in ...,Discrete Perturbation Theory in Analyzing E_7 ...,snarXiv
1,We illustrate that abelian modular forms are r...,Minimal Approaches to the Mu Problem,snarXiv
2,We make contact between Geometric Langlands-du...,Flow Equations in Type IIB Strings Deformed by...,snarXiv
3,"Recently, little work was done on the alternat...",Bounding S-dualities in Chiral Jackiw-Teitelbo...,snarXiv
4,We confirm that gravitational waves in a non-S...,Causality Constraints in Conformal CFTs as To...,snarXiv
5,"From discussing the 1PI effective action, we d...",Models of Squarks,snarXiv
6,"In recent papers, Gaiotto derived that the S-d...",Formulating the Compactification of Line Bundl...,snarXiv
7,Hexaquarks are usually predicted using cosmic ...,Progress in Heavy Ions,snarXiv
8,We demonstrate a deep correspondence between a...,"Models of Positrons, Exploring String Theories...",snarXiv
9,The Lagrangian in models of B-mesons produces ...,Formulating a Check of Primordial Inflation at...,snarXiv


In [22]:
#import json

In [ ]:
#df = pd.read_json('arxiv-metadata-oai-snapshot.json', lines = True)

In [14]:
def get_arXiv_preprints(n_abstracts = 10):
    results = arxiv.Search(
        query = 'cat:hep-th',
        sort_by = 'submittedDate',
        sort_order = 'descending',
        max_results = n_abstracts
    )
    # Some of the results aren't actually in hep-th, so I will remove them
    return [result for result in results if result['arxiv_primary_category']['term']=='hep-th']

# Function to create a dataframe with the arXiv abstracts
def make_arXiv_dataframe(n_abstracts = 10):
    results = get_arXiv_preprints(n_abstracts = n_abstracts)
    arxiv_columns = [
        'id',
        'summary'
    ]
    arxiv_data = pd.DataFrame({key:result[key] for key in arxiv_columns} for result in results).set_index('id')
    arxiv_data['label'] = 'arXiv'
    return arxiv_data

In [15]:
make_arXiv_dataframe()

TypeError: 'Search' object is not iterable

In [13]:
search = arxiv.Search(
  query = 'cat:hep-th',
  max_results = 1000,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print(result.title)

Symmetries and spectral statistics in chaotic conformal field theories II: Maass cusp forms and arithmetic chaos
The ALE Partition Functions of M-Strings
A Semi-classical Spacetime Region with Maximum Entropy
Bayesian location of the QCD critical point from a holographic perspective
Fuzzy Onion as a Matrix Model
Relativistic hydrodynamic fluctuations from an effective action: causality, stability, and the information current
Schwinger-Keldysh effective field theory for stable and causal relativistic hydrodynamics
Non-relativistic limit of the Mielke-Baekler gravity theory
The Nicolai-map approach to supersymmetry
Well-separated soliton-antisoliton pairs with an adjoint Higgs field in 4D space
Multiple Mellin-Barnes integrals and triangulations of point configurations
Kinematic anisotropies and pulsar timing arrays
Regular $(2+1)$-dimensional spatially homogeneous $α'$-corrected black holes in string theory
Electric-Magnetic duality invariance of Galilean relativistic Born-Infeld theory

Non-perturbative instanton effects in the quartic and the sextic double-well potential by the numerical bootstrap approach
Dissipative scattering of spinning black holes at fourth post-Minkowskian order
Relativistic Hydrodynamics under Rotation: Prospects & Limitations from a Holographic Perspective
Cosmological Model with Cosmic transit behaviour in Brans-Dicke Theory
Searching for neutrino-modulino oscillations at the Forward Physics Facility
Angular Momentum Loss Due to Spin-Orbit Effects in the Post-Minkowskian Expansion
Supersymmetric hybrid inflation and metastable cosmic strings in $SU(4)_c \times SU(2)_L \times U(1)_R$
Non-perturbative $\langle φ\rangle$, $\langle φ^2 \rangle$ and the dynamically generated scalar mass with Yukawa interaction in the inflationary de Sitter spacetime
Cosmological Constant and Maximum of Entropy for de Sitter Space
Tree-level Graviton Scattering in the Worldline Formalism
Massless and Partially Massless Limits in Quadratic Gravity
Tidal properties 

A Spin on the Bulk Locality of Slightly Broken Higher Spin Theories
Gapped Interfaces in Fracton Models and Foliated Fields
Raviolo vertex algebras
Gravitational Scattering of Compact Bodies from Worldline Quantum Field Theory
Bulk Action Growth for Holographic Complexity
Non-linear Electrodynamics from Massive Gravity
$\infty$-Bundles
Scattering three closed strings off a Dp-brane in pure spinor formalism
Embedding formalism for ${\mathcal N}$-extended AdS superspace in four dimensions
Pair Production of the Open Superstrings from the Parallel-dressed D3-branes in the Compact Spacetime
Feynman geometry
Homological blocks with simple Lie algebras and Witten--Reshetikhin--Turaev invariants
Big-Bang is a Boundary Condition
Leading all-loop quantum contribution to the effective potential in the inflationary cosmology
One-loop Effective Action up to Dimension Eight: Integrating out Heavy Fermion(s)
Wilson loop correlators at strong coupling in $\mathcal{N}=2$ quiver gauge theories
Ruling o

From tree- to loop-simplicity in affine Toda theories II: higher-order poles and cut decompositions
Krylov Complexity and Spectral Form Factor for Noisy Random Matrix Models
Relativistic Covariance of Scattering
Planar three-loop QCD helicity amplitudes for $V$+jet production at hadron colliders
Supersonic friction of a black hole traversing a self-interacting scalar dark matter cloud
Hamiltonian formalism for Bose excitations in a plasma with a non-Abelian interaction I: plasmon -- hard particle scattering
Novel approach to measure quark/gluon jets at the LHC
Three-dimensional energy-dependent $C$-metric: black hole solutions
Tensions with the flat $\boldsymbolΛ$CDM model from high-redshift cosmography
Fermion states localized on a self-gravitating non-Abelian monopole
Solving two-dimensional adjoint QCD with a basis-function approach
Integrable scattering theory with higher derivative Hamiltonians
Boundary effects on the thermal stability of a black hole at the centre of a conducting

KeyboardInterrupt: 

In [3]:
results_generator = arxiv.Client(
  page_size=1000,
  delay_seconds=3,
  num_retries=3
).results(arxiv.Search(
  query='hep-th',
  max_results = 250,
  id_list=[],
  sort_by=arxiv.SortCriterion.Relevance,
  sort_order=arxiv.SortOrder.Descending,
))

quantum_dots = []
for paper in results_generator:
  # You could do per-paper analysis here; for now, just collect them in a list.
  quantum_dots.append(paper)

In [4]:
qd_df = pd.DataFrame([vars(paper) for paper in quantum_dots])

In [5]:
qd_df

,entry_id,updated,published,title,authors,summary,comment,journal_ref,doi,primary_category,categories,links,pdf_url,_raw
0,http://arxiv.org/abs/hep-th/0611118v1,2006-11-10 12:32:49+00:00,2006-11-10 12:32:49+00:00,Dual Gravitation,[W. Chagas-Filho],We propose a canonical relation between gravit...,None,None,None,hep-th,[hep-th],"[http://arxiv.org/abs/hep-th/0611118v1, http:/...",http://arxiv.org/pdf/hep-th/0611118v1,{'id': 'http://arxiv.org/abs/hep-th/0611118v1'...
1,http://arxiv.org/abs/hep-th/9110012v1,1991-10-01 15:31:00+00:00,1991-10-01 15:31:00+00:00,From factorizable S-matrices to conformal inva...,[Koberle Roland],"Lecture notes on factorizable S-matrices, ther...",46 pages,None,None,hep-th,[hep-th],"[http://arxiv.org/abs/hep-th/9110012v1, http:/...",http://arxiv.org/pdf/hep-th/9110012v1,{'id': 'http://arxiv.org/abs/hep-th/9110012v1'...
2,http://arxiv.org/abs/hep-th/9112002v1,1991-12-03 00:15:51+00:00,1991-12-03 00:15:51+00:00,Homological (ghost) approach to constrained Ha...,[Jim Stasheff],A survey of ghost techniques in mathematical p...,14 pages,None,None,hep-th,"[hep-th, math.QA]","[http://arxiv.org/abs/hep-th/9112002v1, http:/...",http://arxiv.org/pdf/hep-th/9112002v1,{'id': 'http://arxiv.org/abs/hep-th/9112002v1'...
3,http://arxiv.org/abs/hep-th/9112019v1,1991-12-09 20:40:40+00:00,1991-12-09 20:40:40+00:00,An Introduction to 2d Gravity and Solvable Str...,[Emil Martinec],Continuum and discrete approaches to 2d gravit...,29pp. Lectures at the 1991 Trieste Spring School,None,None,hep-th,[hep-th],"[http://arxiv.org/abs/hep-th/9112019v1, http:/...",http://arxiv.org/pdf/hep-th/9112019v1,{'id': 'http://arxiv.org/abs/hep-th/9112019v1'...
4,http://arxiv.org/abs/hep-th/9202091v1,1992-02-28 16:08:40+00:00,1992-02-28 16:08:40+00:00,On a New Class of Integrable Models,"[A. Berkovich, C. Gomez, G. SIERRA]",We define a new class of integrable vertex mod...,16 PAGES,None,10.1142/S0217979292000955,hep-th,[hep-th],"[http://dx.doi.org/10.1142/S0217979292000955, ...",http://arxiv.org/pdf/hep-th/9202091v1,{'id': 'http://arxiv.org/abs/hep-th/9202091v1'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,http://arxiv.org/abs/0909.4424v1,2009-09-24 12:28:02+00:00,2009-09-24 12:28:02+00:00,Three Etudes in QFT,[Dirk Kreimer],We review QFT from the algebraic structure of ...,"6p, contribution to the Int. Congress Math. Ph...",None,10.1142/9789814304634_0054,hep-th,[hep-th],"[http://dx.doi.org/10.1142/9789814304634_0054,...",http://arxiv.org/pdf/0909.4424v1,"{'id': 'http://arxiv.org/abs/0909.4424v1', 'gu..."
246,http://arxiv.org/abs/0910.0621v1,2009-10-04 18:26:37+00:00,2009-10-04 18:26:37+00:00,Solving field equations in spinor electrodynamics,[A. V. Bratchikov],Solutions of classical and quantum equations o...,8 pages,None,None,hep-th,[hep-th],"[http://arxiv.org/abs/0910.0621v1, http://arxi...",http://arxiv.org/pdf/0910.0621v1,"{'id': 'http://arxiv.org/abs/0910.0621v1', 'gu..."
247,http://arxiv.org/abs/0911.3692v1,2009-11-19 02:54:47+00:00,2009-11-19 02:54:47+00:00,Lagrangian with U(1)-SU(2) mixing,[Bernd A. Berg],Principal axis transformation is performed for...,2 pages,None,None,hep-th,[hep-th],"[http://arxiv.org/abs/0911.3692v1, http://arxi...",http://arxiv.org/pdf/0911.3692v1,"{'id': 'http://arxiv.org/abs/0911.3692v1', 'gu..."
248,http://arxiv.org/abs/0912.4717v2,2010-03-01 15:41:16+00:00,2009-12-23 19:30:42+00:00,Curved Space (Matrix) Membranes,[Jens Hoppe],Hamiltonian formulations of M-branes moving in...,None,None,10.1007/s10714-010-1017-1,hep-th,[hep-th],"[http://dx.doi.org/10.1007/s10714-010-1017-1, ...",http://arxiv.org/pdf/0912.4717v2,"{'id': 'http://arxiv.org/abs/0912.4717v2', 'gu..."


In [12]:
results_generator = arxiv.Client(
  page_size=1000,
  delay_seconds=3,
  num_retries=3
).results(arxiv.Search(
  query='hep-ph',
  max_results = 250,
  id_list=[],
  sort_by=arxiv.SortCriterion.Relevance,
  sort_order=arxiv.SortOrder.Descending,
))

ph = []
for paper in results_generator:
  # You could do per-paper analysis here; for now, just collect them in a list.
  ph.append(paper)
    
ph_df = pd.DataFrame([vars(paper) for paper in ph])

ph_df

,entry_id,updated,published,title,authors,summary,comment,journal_ref,doi,primary_category,categories,links,pdf_url,_raw
0,http://arxiv.org/abs/hep-ph/9205226v1,1992-05-20 12:38:37+00:00,1992-05-20 12:38:37+00:00,Recent Developments in Superstring Phenomenology,[Jan Louis],Recent developments in superstring phenomenolo...,13 pages,None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/hep-ph/9205226v1, http:/...",http://arxiv.org/pdf/hep-ph/9205226v1,{'id': 'http://arxiv.org/abs/hep-ph/9205226v1'...
1,http://arxiv.org/abs/hep-ph/9210236v2,1992-10-15 19:36:59+00:00,1992-10-14 20:56:13+00:00,The SSC: Programme and Searches for New Particles,[Benjamin Grinstein],Talk presented at the Trieste Workshop on the ...,8 p,None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/hep-ph/9210236v2, http:/...",http://arxiv.org/pdf/hep-ph/9210236v2,{'id': 'http://arxiv.org/abs/hep-ph/9210236v2'...
2,http://arxiv.org/abs/hep-ph/9211298v1,1992-11-24 01:51:09+00:00,1992-11-24 01:51:09+00:00,Conference Summary,[Steven Weinberg],Talk presented at the XXVI International Confe...,40 pages,"AIP Conf.Proc.272:346-366,1993",10.1063/1.43503,hep-ph,[hep-ph],"[http://dx.doi.org/10.1063/1.43503, http://arx...",http://arxiv.org/pdf/hep-ph/9211298v1,{'id': 'http://arxiv.org/abs/hep-ph/9211298v1'...
3,http://arxiv.org/abs/hep-ph/9212220v2,1992-12-06 13:16:00+00:00,1992-12-03 20:55:00+00:00,Radiative Decays of the P-Wave Charmed Mesons,"[J. G. Korner, D. Pirjol, K. Schilcher]",Minor (mainly numerical) corrections.,"12 pages, LaTeX, MZ-TH/92-51",Phys.Rev. D47 (1993) 3955-3961,10.1103/PhysRevD.47.3955,hep-ph,[hep-ph],"[http://dx.doi.org/10.1103/PhysRevD.47.3955, h...",http://arxiv.org/pdf/hep-ph/9212220v2,{'id': 'http://arxiv.org/abs/hep-ph/9212220v2'...
4,http://arxiv.org/abs/hep-ph/9303290v1,1993-03-22 22:26:32+00:00,1993-03-22 22:26:32+00:00,Exotic Fermions,[David London],I review the current limits on the mixing of o...,"23 pages, LaTeX, to be published in ""Precision...",None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/hep-ph/9303290v1, http:/...",http://arxiv.org/pdf/hep-ph/9303290v1,{'id': 'http://arxiv.org/abs/hep-ph/9303290v1'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,http://arxiv.org/abs/1310.3292v1,2013-10-11 21:03:33+00:00,2013-10-11 21:03:33+00:00,Theory Summary,[Fabio Zwirner],Talk given at the XLVIII Rencontres de Moriond...,"16 pages, 7 figures",None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/1310.3292v1, http://arxi...",http://arxiv.org/pdf/1310.3292v1,"{'id': 'http://arxiv.org/abs/1310.3292v1', 'gu..."
246,http://arxiv.org/abs/1312.5500v1,2013-12-19 12:09:12+00:00,2013-12-19 12:09:12+00:00,On Vector Dominance,[V. A. Petrov],We argue that a Regge-pole modification of the...,4 pages,None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/1312.5500v1, http://arxi...",http://arxiv.org/pdf/1312.5500v1,"{'id': 'http://arxiv.org/abs/1312.5500v1', 'gu..."
247,http://arxiv.org/abs/1406.5379v1,2014-06-20 13:28:11+00:00,2014-06-20 13:28:11+00:00,QCD and High Energy Interactions: Moriond 2014...,[Thomas Gehrmann],This article summarizes new theoretical develo...,7 pages,None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/1406.5379v1, http://arxi...",http://arxiv.org/pdf/1406.5379v1,"{'id': 'http://arxiv.org/abs/1406.5379v1', 'gu..."
248,http://arxiv.org/abs/1410.7142v1,2014-10-27 07:36:21+00:00,2014-10-27 07:36:21+00:00,Revisiting texture 5 zero quark mass matrices,[Priyanka Fakay],The question of viability of texture 5 zero Fr...,"4 pages, 2 figures",None,None,hep-ph,[hep-ph],"[http://arxiv.org/abs/1410.7142v1, http://arxi...",http://arxiv.org/pdf/1410.7142v1,"{'id': 'http://arxiv.org/abs/1410.7142v1', 'gu..."


In [13]:
th_df = qd_df[['title','summary']]

In [14]:
th_df['label'] = 'arXiv'

/Users/shawnyou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
th_df

,title,summary,label
0,Dual Gravitation,We propose a canonical relation between gravit...,arXiv
1,From factorizable S-matrices to conformal inva...,"Lecture notes on factorizable S-matrices, ther...",arXiv
2,Homological (ghost) approach to constrained Ha...,A survey of ghost techniques in mathematical p...,arXiv
3,An Introduction to 2d Gravity and Solvable Str...,Continuum and discrete approaches to 2d gravit...,arXiv
4,On a New Class of Integrable Models,We define a new class of integrable vertex mod...,arXiv
...,...,...,...
245,Three Etudes in QFT,We review QFT from the algebraic structure of ...,arXiv
246,Solving field equations in spinor electrodynamics,Solutions of classical and quantum equations o...,arXiv
247,Lagrangian with U(1)-SU(2) mixing,Principal axis transformation is performed for...,arXiv
248,Curved Space (Matrix) Membranes,Hamiltonian formulations of M-branes moving in...,arXiv


In [16]:
ph_df = ph_df[['title', 'summary']]
ph_df['label'] = 'arXiv'
ph_df

/Users/shawnyou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,summary,label
0,Recent Developments in Superstring Phenomenology,Recent developments in superstring phenomenolo...,arXiv
1,The SSC: Programme and Searches for New Particles,Talk presented at the Trieste Workshop on the ...,arXiv
2,Conference Summary,Talk presented at the XXVI International Confe...,arXiv
3,Radiative Decays of the P-Wave Charmed Mesons,Minor (mainly numerical) corrections.,arXiv
4,Exotic Fermions,I review the current limits on the mixing of o...,arXiv
...,...,...,...
245,Theory Summary,Talk given at the XLVIII Rencontres de Moriond...,arXiv
246,On Vector Dominance,We argue that a Regge-pole modification of the...,arXiv
247,QCD and High Energy Interactions: Moriond 2014...,This article summarizes new theoretical develo...,arXiv
248,Revisiting texture 5 zero quark mass matrices,The question of viability of texture 5 zero Fr...,arXiv


In [19]:
arxiv_df = pd.concat([th_df, ph_df])

In [20]:
arxiv_df

,title,summary,label
0,Dual Gravitation,We propose a canonical relation between gravit...,arXiv
1,From factorizable S-matrices to conformal inva...,"Lecture notes on factorizable S-matrices, ther...",arXiv
2,Homological (ghost) approach to constrained Ha...,A survey of ghost techniques in mathematical p...,arXiv
3,An Introduction to 2d Gravity and Solvable Str...,Continuum and discrete approaches to 2d gravit...,arXiv
4,On a New Class of Integrable Models,We define a new class of integrable vertex mod...,arXiv
...,...,...,...
245,Theory Summary,Talk given at the XLVIII Rencontres de Moriond...,arXiv
246,On Vector Dominance,We argue that a Regge-pole modification of the...,arXiv
247,QCD and High Energy Interactions: Moriond 2014...,This article summarizes new theoretical develo...,arXiv
248,Revisiting texture 5 zero quark mass matrices,The question of viability of texture 5 zero Fr...,arXiv


In [23]:
df = pd.concat([test, arxiv_df])

In [24]:
df

,summary,title,label
0,We make contact with particle fluctuations in ...,Discrete Perturbation Theory in Analyzing E_7 ...,snarXiv
1,We illustrate that abelian modular forms are r...,Minimal Approaches to the Mu Problem,snarXiv
2,We make contact between Geometric Langlands-du...,Flow Equations in Type IIB Strings Deformed by...,snarXiv
3,"Recently, little work was done on the alternat...",Bounding S-dualities in Chiral Jackiw-Teitelbo...,snarXiv
4,We confirm that gravitational waves in a non-S...,Causality Constraints in Conformal CFTs as To...,snarXiv
...,...,...,...
245,Talk given at the XLVIII Rencontres de Moriond...,Theory Summary,arXiv
246,We argue that a Regge-pole modification of the...,On Vector Dominance,arXiv
247,This article summarizes new theoretical develo...,QCD and High Energy Interactions: Moriond 2014...,arXiv
248,The question of viability of texture 5 zero Fr...,Revisiting texture 5 zero quark mass matrices,arXiv


In [25]:
df.to_csv('dataset1000.csv', index = False)

In [3]:
df = pd.read_csv('dataset1000.csv')

In [4]:
df

,summary,title,label
0,We make contact with particle fluctuations in ...,Discrete Perturbation Theory in Analyzing E_7 ...,snarXiv
1,We illustrate that abelian modular forms are r...,Minimal Approaches to the Mu Problem,snarXiv
2,We make contact between Geometric Langlands-du...,Flow Equations in Type IIB Strings Deformed by...,snarXiv
3,"Recently, little work was done on the alternat...",Bounding S-dualities in Chiral Jackiw-Teitelbo...,snarXiv
4,We confirm that gravitational waves in a non-S...,Causality Constraints in Conformal CFTs as To...,snarXiv
...,...,...,...
995,Talk given at the XLVIII Rencontres de Moriond...,Theory Summary,arXiv
996,We argue that a Regge-pole modification of the...,On Vector Dominance,arXiv
997,This article summarizes new theoretical develo...,QCD and High Energy Interactions: Moriond 2014...,arXiv
998,The question of viability of texture 5 zero Fr...,Revisiting texture 5 zero quark mass matrices,arXiv


In [7]:
# pre-processing
# check out nfx
import neattext.functions as nfx

In [8]:
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [10]:
df['summary'] = df['summary'].apply(nfx.remove_stopwords)

In [13]:
df['title'] = df['title'].apply(nfx.remove_stopwords)

In [14]:
df

,summary,title,label
0,"contact particle fluctuations early universe, ...",Discrete Perturbation Theory Analyzing E_7 N=n...,snarXiv
1,illustrate abelian modular forms related B-typ...,Minimal Approaches Mu Problem,snarXiv
2,contact Geometric Langlands-duality JT gravity...,Flow Equations Type IIB Strings Deformed 'T Ho...,snarXiv
3,"Recently, little work alternative O(5) Model w...",Bounding S-dualities Chiral Jackiw-Teitelboim ...,snarXiv
4,confirm gravitational waves non-Sp(3) model mu...,Causality Constraints Conformal CFTs Topologic...,snarXiv
...,...,...,...
995,"Talk given XLVIII Rencontres de Moriond, Elect...",Theory Summary,arXiv
996,argue Regge-pole modification VDM puts equal f...,Vector Dominance,arXiv
997,article summarizes new theoretical development...,QCD High Energy Interactions: Moriond 2014 The...,arXiv
998,question viability texture 5 zero Fritzsch-lik...,Revisiting texture 5 zero quark mass matrices,arXiv


In [15]:
df['summary'] = df['summary'].apply(nfx.remove_punctuations)
df['title'] = df['title'].apply(nfx.remove_punctuations)

In [16]:
df

,summary,title,label
0,contact particle fluctuations early universe a...,Discrete Perturbation Theory Analyzing E7 N=n ...,snarXiv
1,illustrate abelian modular forms related Btype...,Minimal Approaches Mu Problem,snarXiv
2,contact Geometric Langlandsduality JT gravity ...,Flow Equations Type IIB Strings Deformed T Hoo...,snarXiv
3,Recently little work alternative O(5) Model wa...,Bounding Sdualities Chiral JackiwTeitelboim Gr...,snarXiv
4,confirm gravitational waves nonSp(3) model mut...,Causality Constraints Conformal CFTs Topologic...,snarXiv
...,...,...,...
995,Talk given XLVIII Rencontres de Moriond Electr...,Theory Summary,arXiv
996,argue Reggepole modification VDM puts equal fo...,Vector Dominance,arXiv
997,article summarizes new theoretical development...,QCD High Energy Interactions: Moriond 2014 The...,arXiv
998,question viability texture 5 zero Fritzschlike...,Revisiting texture 5 zero quark mass matrices,arXiv


In [17]:
df.to_csv('df1000_cleaned.csv', index = False)

In [2]:
########
########
# lemmatize
#

In [15]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shawnyou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv('df1000_cleaned.csv')

In [8]:
df

,summary,title,label
0,contact particle fluctuations early universe a...,Discrete Perturbation Theory Analyzing E7 N=n ...,snarXiv
1,illustrate abelian modular forms related Btype...,Minimal Approaches Mu Problem,snarXiv
2,contact Geometric Langlandsduality JT gravity ...,Flow Equations Type IIB Strings Deformed T Hoo...,snarXiv
3,Recently little work alternative O(5) Model wa...,Bounding Sdualities Chiral JackiwTeitelboim Gr...,snarXiv
4,confirm gravitational waves nonSp(3) model mut...,Causality Constraints Conformal CFTs Topologic...,snarXiv
...,...,...,...
995,Talk given XLVIII Rencontres de Moriond Electr...,Theory Summary,arXiv
996,argue Reggepole modification VDM puts equal fo...,Vector Dominance,arXiv
997,article summarizes new theoretical development...,QCD High Energy Interactions: Moriond 2014 The...,arXiv
998,question viability texture 5 zero Fritzschlike...,Revisiting texture 5 zero quark mass matrices,arXiv


In [5]:
print(df.isnull().values.any())
print(df.isnull().sum())

True
summary    0
title      1
label      0
dtype: int64


In [9]:
df = df.dropna()

In [7]:
df.to_csv('df1000_cleaned.csv', index = False)

In [10]:
df

,summary,title,label
0,contact particle fluctuations early universe a...,Discrete Perturbation Theory Analyzing E7 N=n ...,snarXiv
1,illustrate abelian modular forms related Btype...,Minimal Approaches Mu Problem,snarXiv
2,contact Geometric Langlandsduality JT gravity ...,Flow Equations Type IIB Strings Deformed T Hoo...,snarXiv
3,Recently little work alternative O(5) Model wa...,Bounding Sdualities Chiral JackiwTeitelboim Gr...,snarXiv
4,confirm gravitational waves nonSp(3) model mut...,Causality Constraints Conformal CFTs Topologic...,snarXiv
...,...,...,...
995,Talk given XLVIII Rencontres de Moriond Electr...,Theory Summary,arXiv
996,argue Reggepole modification VDM puts equal fo...,Vector Dominance,arXiv
997,article summarizes new theoretical development...,QCD High Energy Interactions: Moriond 2014 The...,arXiv
998,question viability texture 5 zero Fritzschlike...,Revisiting texture 5 zero quark mass matrices,arXiv


In [12]:
df = df.apply(lambda x: x.astype(str).str.lower())

In [13]:
df

,summary,title,label
0,contact particle fluctuations early universe a...,discrete perturbation theory analyzing e7 n=n ...,snarxiv
1,illustrate abelian modular forms related btype...,minimal approaches mu problem,snarxiv
2,contact geometric langlandsduality jt gravity ...,flow equations type iib strings deformed t hoo...,snarxiv
3,recently little work alternative o(5) model wa...,bounding sdualities chiral jackiwteitelboim gr...,snarxiv
4,confirm gravitational waves nonsp(3) model mut...,causality constraints conformal cfts topologic...,snarxiv
...,...,...,...
995,talk given xlviii rencontres de moriond electr...,theory summary,arxiv
996,argue reggepole modification vdm puts equal fo...,vector dominance,arxiv
997,article summarizes new theoretical development...,qcd high energy interactions: moriond 2014 the...,arxiv
998,question viability texture 5 zero fritzschlike...,revisiting texture 5 zero quark mass matrices,arxiv


In [16]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [17]:
df['summary'] = df['summary'].map(lemmatize_text)

In [18]:
df['title'] = df['title'].map(lemmatize_text)

In [19]:
df

,summary,title,label
0,contact particle fluctuation early universe ac...,discrete perturbation theory analyzing e7 n=n ...,snarxiv
1,illustrate abelian modular form related btype ...,minimal approach mu problem,snarxiv
2,contact geometric langlandsduality jt gravity ...,flow equation type iib string deformed t hooft...,snarxiv
3,recently little work alternative o(5) model wa...,bounding sdualities chiral jackiwteitelboim gr...,snarxiv
4,confirm gravitational wave nonsp(3) model mutu...,causality constraint conformal cfts topologica...,snarxiv
...,...,...,...
995,talk given xlviii rencontres de moriond electr...,theory summary,arxiv
996,argue reggepole modification vdm put equal foo...,vector dominance,arxiv
997,article summarizes new theoretical development...,qcd high energy interactions: moriond 2014 the...,arxiv
998,question viability texture 5 zero fritzschlike...,revisiting texture 5 zero quark mass matrix,arxiv


In [22]:
df.to_csv('df1000_cleaned.csv', index = False)

In [23]:
df

,summary,title,label
0,contact particle fluctuation early universe ac...,discrete perturbation theory analyzing e7 n=n ...,snarxiv
1,illustrate abelian modular form related btype ...,minimal approach mu problem,snarxiv
2,contact geometric langlandsduality jt gravity ...,flow equation type iib string deformed t hooft...,snarxiv
3,recently little work alternative o(5) model wa...,bounding sdualities chiral jackiwteitelboim gr...,snarxiv
4,confirm gravitational wave nonsp(3) model mutu...,causality constraint conformal cfts topologica...,snarxiv
...,...,...,...
995,talk given xlviii rencontres de moriond electr...,theory summary,arxiv
996,argue reggepole modification vdm put equal foo...,vector dominance,arxiv
997,article summarizes new theoretical development...,qcd high energy interactions: moriond 2014 the...,arxiv
998,question viability texture 5 zero fritzschlike...,revisiting texture 5 zero quark mass matrix,arxiv


In [26]:
df['title']

0      Discrete Perturbation Theory Analyzing E7 N=n ...
1                          Minimal Approaches Mu Problem
2      Flow Equations Type IIB Strings Deformed T Hoo...
3      Bounding Sdualities Chiral JackiwTeitelboim Gr...
4      Causality Constraints Conformal CFTs Topologic...
                             ...                        
995                                       Theory Summary
996                                     Vector Dominance
997    QCD High Energy Interactions: Moriond 2014 The...
998        Revisiting texture 5 zero quark mass matrices
999                             Proton charge extensions
Name: title, Length: 1000, dtype: object

In [28]:
df.dtypes

summary    object
title      object
label      object
dtype: object

In [3]:
df = pd.read_csv('df1000_cleaned.csv')

In [4]:
df

,summary,title,label
0,contact particle fluctuation early universe ac...,discrete perturbation theory analyzing e7 n=n ...,snarxiv
1,illustrate abelian modular form related btype ...,minimal approach mu problem,snarxiv
2,contact geometric langlandsduality jt gravity ...,flow equation type iib string deformed t hooft...,snarxiv
3,recently little work alternative o(5) model wa...,bounding sdualities chiral jackiwteitelboim gr...,snarxiv
4,confirm gravitational wave nonsp(3) model mutu...,causality constraint conformal cfts topologica...,snarxiv
...,...,...,...
994,talk given xlviii rencontres de moriond electr...,theory summary,arxiv
995,argue reggepole modification vdm put equal foo...,vector dominance,arxiv
996,article summarizes new theoretical development...,qcd high energy interactions: moriond 2014 the...,arxiv
997,question viability texture 5 zero fritzschlike...,revisiting texture 5 zero quark mass matrix,arxiv


In [5]:
df['title'] = df['title'].str.replace(r'\(|\)', '')

/Users/shawnyou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [6]:
df = df.drop_duplicates(subset = ['title']).reset_index()
df = df[['title', 'label']]

test_snarxiv = df.iloc[:100]
test_arxiv = df.iloc[-100:]
training_df = df.iloc[100:-100]

In [8]:
training_df

,title,label
100,reformulating nelsonweinberg rs1,snarxiv
101,probe certain notion integrability check black...,snarxiv
102,heterotic string theory excluding path integra...,snarxiv
103,entropic approach lithium problem,snarxiv
104,model kaon v tdual technicolor,snarxiv
...,...,...
884,neutrino fluctuating plasma solar neutrino puzzle,arxiv
885,extremal size fluctuation qcd dipole cascading,arxiv
886,nature greatest puzzle,arxiv
887,relation hadron mass,arxiv


In [10]:
training_df.to_csv('training_data.csv', index = False)

In [11]:
testing_df = pd.concat([test_snarxiv, test_arxiv], axis = 0)

In [12]:
testing_df

,title,label
0,discrete perturbation theory analyzing e7 n=n ...,snarxiv
1,minimal approach mu problem,snarxiv
2,flow equation type iib string deformed t hooft...,snarxiv
3,bounding sdualities chiral jackiwteitelboim gr...,snarxiv
4,causality constraint conformal cfts topologica...,snarxiv
...,...,...
984,theory summary,arxiv
985,vector dominance,arxiv
986,qcd high energy interactions: moriond 2014 the...,arxiv
987,revisiting texture 5 zero quark mass matrix,arxiv


In [13]:
testing_df.to_csv('testing_data.csv', index = False)